In [1]:
import pandas as pd

import numpy as np

import xlsxwriter, os

import matplotlib.pyplot as plt

%matplotlib inline

from openpyxl import load_workbook

from sklearn import preprocessing

In [2]:
# save sales by industry into excel files

def create_excel_fnc(path, df, sheetname):
    
    writer = pd.ExcelWriter(path)
    
    df.to_excel(writer, sheetname)
    
    writer.save()
    
def add_sheet_fnc(path, df,sheetname):
    
    book = load_workbook(path)
    
    writer = pd.ExcelWriter(path, engine='openpyxl')
    
    writer.book = book
    
    df.to_excel(writer, sheet_name=sheetname)
    
    writer.save()
    
    writer.close()

# Process MN Industry 2015 Dataset

In [3]:
# load MN Industry Dataset

MN_df = pd.read_excel(os.getcwd()+r'\MN_Data\MN_Industry2015.xls')

In [4]:
# drop unexpected rows

MN_df = MN_df[MN_df.COUNTY != 'NON-MINNESOTA CO']

In [5]:
# extract all industry names

Industry_Names = MN_df.INDUSTRY.unique()

In [7]:
Industry_Names[0]

'236 CONSTRUCT -BUILDINGS'

In [6]:
Industry_Names[33] = '999 UNDESIGNATED or SUPPRESSED'

In [8]:
Construct_df = MN_df[MN_df.INDUSTRY == Industry_Names[0]]
    
Construct_County = Construct_df.COUNTY.tolist()

In [9]:
Construct_County_Sales = Construct_df[['COUNTY', 'GROSS SALES']]

In [35]:
Construct_County_Sales.set_index('COUNTY', inplace=True)

In [38]:
Construct_County_Sales = Construct_County_Sales.astype('float64')

# Load our raw data by County

In [10]:
# load our data

Raw_Data = pd.ExcelFile(r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Raw_Data.xlsx')

# Process Indices

In [11]:
sheetnames = Raw_Data.sheet_names

sheetnames.remove('Nat_Frn_Popn')

indices_names = sheetnames[0:8]

In [40]:
def gen_indices_data(path_raw, path_corr, sheetname, raw_data):
    
    df = raw_data.parse(sheetname)
    
    df.set_index('County', inplace=True)
    
    df = df.loc[Construct_County] # need to change for each industry
    
    add_sheet_fnc(path_raw, df, sheetname)
    
    corr_df = pd.DataFrame()
    
    # need to change path for each Industry

    corr_df['Correlation Coefficient'] = df[df.columns[:]].apply(lambda x: x.corr(Construct_County_Sales['GROSS SALES']))

    add_sheet_fnc(path_corr, corr_df, sheetname)

In [41]:
sheetnames = Raw_Data.sheet_names

sheetnames.remove('Nat_Frn_Popn')

indices_names = sheetnames[0:8]

In [42]:
# need to change path for each Industry

path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Raw_Data\Construct\Indices.xlsx'

path2 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Correlation_Data\Construct\Indices_Corr.xlsx'

for sheetname in indices_names:
    
    gen_indices_data(path1, path2, sheetname, Raw_Data)

# Process Category Dataset

In [47]:
def create_Categories_Data(path_raw, path_corr, raw_data, sheetname):

    df = raw_data.parse(sheetname)
    
    variable_names = df.loc[0][1:]

    df.set_index('County', inplace=True)
    
    df = df.loc[Construct_County]
    
    raw_df = df.reset_index()

    raw_df.loc[-1] = variable_names

    raw_df.index = raw_df.index+1

    raw_df = raw_df.sort_index()
    
    df = df.astype('float64')

    add_sheet_fnc(path_raw, raw_df,sheetname)
    
    corr_df = pd.DataFrame()
    
    corr_df['Variable Name'] = variable_names
    
    # need to change for each Industry

    corr_df['Correlation Coefficient'] = df[df.columns[:]].apply(lambda x: x.corr(Construct_County_Sales['GROSS SALES']))

    add_sheet_fnc(path_corr, corr_df, sheetname)

In [48]:
category_names = sheetnames[9:]

In [49]:
# need to change path for each Industry

path1 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Raw_Data\Construct\Categories.xlsx'

path2 = r'C:\Users\PL_Dell3668_One\Desktop\MN_Industry\Corr_Industry\Industry_Correlation_Data\Construct\Categories_Corr.xlsx'

In [50]:
for sheetname in category_names:
    
    create_Categories_Data(path1, path2, Raw_Data, sheetname)